In [ ]:
import xarray
import numpy as np

import hvplot.xarray

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geoviews as gv

import matplotlib.pyplot as plt
%matplotlib inline
import cmocean.cm as cmo

In [ ]:
import os
home_top_dir=os.environ["HOME"]
print("Assuming your files are stored somewhere similar to mine,\nwith $HOME/COAWST and $HOME/seahorce-scidac/ROMSX\nwhere your environment variable $HOME reads as "+home_top_dir+"\n")
print(home_top_dir)

Load ROMS data from upwelling test
==================================

In [ ]:
#ds_roms = xarray.open_dataset(home_top_dir+"/COAWST/roms_his.nc", chunks={"ocean_time": 10})
#ds_roms
ds_roms = xarray.open_dataset(home_top_dir+"/roms_his.nc", chunks={"ocean_time": 10})
ds_roms

In [ ]:

if ds_roms.Vtransform == 1:
    Zo_rho = ds_roms.hc * (ds_roms.s_rho - ds_roms.Cs_r) + ds_roms.Cs_r * ds_roms.h
    z_rho = Zo_rho + ds_roms.zeta * (1 + Zo_rho / ds_roms.h)
    Zo_w = ds_roms.hc * (ds_roms.s_w - ds_roms.Cs_w) + ds_roms.Cs_w * ds_roms.h
    z_w = Zo_w + ds_roms.zeta * (1 + Zo_w / ds_roms.h)
    # also include z coordinates with mean sea level (constant over time)
    z_rho0 = Zo_rho
    z_w0 = Zo_w
elif ds_roms.Vtransform == 2:
    Zo_rho = (ds_roms.hc * ds_roms.s_rho + ds_roms.Cs_r * ds_roms.h) / (ds_roms.hc + ds_roms.h)
    z_rho = ds_roms.zeta + (ds_roms.zeta + ds_roms.h) * Zo_rho
    Zo_w = (ds_roms.hc * ds_roms.s_w + ds_roms.Cs_w * ds_roms.h) / (ds_roms.hc + ds_roms.h)
    z_w = ds_roms.zeta + (ds_roms.zeta + ds_roms.h) * Zo_w
    # also include z coordinates with mean sea level (constant over time)
    z_rho0 = ds_roms.h * Zo_rho
    z_w0 = ds_roms.h * Zo_w

ds_roms.coords["z_w"] = z_w.transpose('ocean_time', 's_w', 'eta_rho', 'xi_rho')
ds_roms.coords["z_rho"] = z_rho.transpose('ocean_time', 's_rho', 'eta_rho', 'xi_rho')
ds_roms.coords["z_rho0"] = z_rho0.transpose('s_rho', 'eta_rho', 'xi_rho')
ds_roms.coords["z_w0"] = z_w0.transpose('s_w', 'eta_rho', 'xi_rho')



In [ ]:
section = ds_roms.temp.isel(eta_rho=0, ocean_time=100)
plt=section.plot(x="xi_rho", y="z_rho", figsize=(15, 6), clim=(25, 35))
plt.set_clim([14, 22]);

In [ ]:
print(ds_roms.temp.isel(xi_rho=slice(1,2),eta_rho=slice(1,2),s_rho=slice(11,12),ocean_time=0).max().as_numpy())
print(ds_roms.temp.isel(xi_rho=slice(1,2),eta_rho=slice(1,2),s_rho=slice(11,12),ocean_time=0).min().as_numpy())
print(ds_roms.z_rho0.as_numpy())

Load ROMSX dataset from Exec/Upwelling
======================================

In [ ]:
#ds = xarray.open_dataset(home_top_dir+"/seahorce-scidac/ROMSX/Exec/Upwelling/plt_his.nc", chunks={"ocean_time": 2})
#ds
ds = xarray.open_dataset(home_top_dir+"/plt_his.nc", chunks={"ocean_time": 2})
ds
#section=ds.x_velocity.isel(ocean_time=0,s_rho=slice(0,15),eta_rho=slice(1,80),xi_rho=slice(1,40))
section=ds.x_velocity.isel(ocean_time=0,s_rho=slice(0,15),eta_rho=40,xi_rho=slice(1,40))
print(np.sum(section.as_numpy()))
#print((ds_roms.u.isel(ocean_time=0,s_rho=slice(0,15),eta_u=40,xi_u=slice(1,40)).as_numpy())-(ds.x_velocity.isel(ocean_time=0,s_rho=slice(0,15),eta_rho=40,xi_rho=slice(2,41))).as_numpy())
##plt=section.plot(x="x_rho", y="z_rho", figsize=(15, 6), clim=(25, 35))

##print((ds_roms.u.isel(ocean_time=0,s_rho=3,eta_u=3,xi_u=3).as_numpy()))
#print((ds.x_velocity.isel(ocean_time=1,s_rho=2,eta_rho=3,xi_rho=3).as_numpy()))
#print((ds_roms.u.isel(ocean_time=1,s_rho=2,eta_u=2,xi_u=2)).as_numpy())

print((ds_roms.u.isel(ocean_time=1,s_rho=2,eta_u=2,xi_u=2)).as_numpy()-(ds.x_velocity.isel(ocean_time=1,s_rho=2,eta_rho=2,xi_rho=3).as_numpy()))
#print((ds_roms.u.isel(ocean_time=1,s_rho=2,eta_u=2,xi_u=slice(2,40))).as_numpy()-(ds.x_velocity.isel(ocean_time=1,s_rho=2,eta_rho=2,xi_rho=slice(3,41)).as_numpy()))
print((ds_roms.u.isel(ocean_time=1,s_rho=2,eta_u=3,xi_u=5)).as_numpy()-(ds.x_velocity.isel(ocean_time=1,s_rho=2,eta_rho=3,xi_rho=6).as_numpy()))
print((ds_roms.v.isel(ocean_time=1,s_rho=2,eta_v=2,xi_v=2)).as_numpy()-(ds.y_velocity.isel(ocean_time=1,s_rho=2,eta_rho=3,xi_rho=2).as_numpy()))
print((ds_roms.v.isel(ocean_time=1,s_rho=2,eta_v=3,xi_v=5)).as_numpy()-(ds.y_velocity.isel(ocean_time=1,s_rho=2,eta_rho=4,xi_rho=5).as_numpy()))



In [ ]:
section = ds.temp.isel(eta_rho=40,ocean_time=1)
plt=section.plot(x="x_rho", y="z_rho", figsize=(15, 6), clim=(25, 35))
plt.set_clim([14, 22]);

In [ ]:
print(ds.temp.isel(xi_rho=slice(1,2),eta_rho=slice(1,2),s_rho=slice(11,12)).max().as_numpy())
print(ds.temp.isel(xi_rho=slice(1,2),eta_rho=slice(1,2),s_rho=slice(11,12)).min().as_numpy())
print(ds.z_rho.as_numpy())
print(ds.s_rho.as_numpy())
print(ds.z_rho.isel(xi_rho=slice(0,2),eta_rho=slice(0,2),s_rho=slice(0,16)).as_numpy())

Compare
=======
Goal is to be able to make an "error" plot for more eye-norms, next step requires making the coordinate directions the same such that xi_rho (the 2-d version) and x_rho (the 1-d version) are comparable

In [ ]:
#print(ds.temp.isel(xi_rho=slice(2,3),eta_rho=slice(3,4)).as_numpy())
for k in np.arange(-1,15):
    a=(ds.temp.isel(xi_rho=slice(1,2), eta_rho=slice(1,2),s_rho=slice(1+k,2+k),ocean_time=0)).as_numpy()
    b=(ds_roms.temp.isel(xi_rho=slice(1,2), eta_rho=slice(1,2),s_rho=slice(1+k,2+k), ocean_time=0)).as_numpy()
#    print("ROMSX:       ")
#    print(a[0][0][0])
#    print("ROMS:        ")
#    print(b[0][0][0])
    diff_temp=a-b
#    print("ROMSX-ROMS:  ")
#    print(diff_temp)
    if(diff_temp!=0.0):
        print("sdlfkjdlkfjsl")
        print(diff_temp-0.0)


In [ ]:
diff_temp=((ds_roms.u.isel(ocean_time=100)).as_numpy()-(ds_roms.u.isel(ocean_time=0)).as_numpy())
diff_temp.plot()
diff_temp.sum().as_numpy()

In [ ]:
diff_temp=((ds_roms.u.isel(ocean_time=100)).as_numpy()-(ds_roms.u.isel(ocean_time=0)).as_numpy())
diff_temp.plot()
diff_temp.sum().as_numpy()

In [ ]:

#section = ds_roms.w.isel(eta_rho=40,ocean_time=100)-ds_roms.w.isel(eta_rho=40, ocean_time=0)
section = ds_roms.salt.isel(eta_rho=40,ocean_time=100)
plt=section.plot(x="xi_rho", y="s_rho", figsize=(15, 6), clim=(25, 35))
#plt.set_clim([14, 22]);

In [ ]:

#section = ds_roms.w.isel(eta_rho=40,ocean_time=100)-ds_roms.w.isel(eta_rho=40, ocean_time=0)
section = ds_roms.w.isel(eta_rho=40,ocean_time=100)
plt=section.plot(x="x_rho", y="s_w", figsize=(15, 6), clim=(25, 35))
#plt.set_clim([14, 22]);


In [ ]:

#section = ds_roms.w.isel(eta_rho=40,ocean_time=100)-ds_roms.w.isel(eta_rho=40, ocean_time=0)
section = ds_roms.v.isel(eta_v=40,ocean_time=100)
plt=section.plot(x="xi_v", y="s_rho", figsize=(15, 6), clim=(25, 35))
#plt.set_clim([14, 22]);

In [ ]:

section = ds.temp.isel(ocean_time=0)-ds_roms.temp.isel(ocean_time=0)
print(section.as_numpy().max())
print(section.as_numpy().min())
#plt.set_clim([14, 22]);
